<a href="https://colab.research.google.com/github/jsykes50/accounting-python/blob/main/Accounting%20Sales%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sales Analysis Project Code
import os
import pandas as pd
from common_code import data_folder
import seaborn as sns

# Preparing the data - Product and Cost data:
products_df = pd.read_csv(f'{data_folder}/products.csv')

products_df.head()

products_df.info()

Brand = pd.read_csv(f'{data_folder}/products.csv')
products_df = products_df.drop('Brand', axis='columns')

products_df.head()

costs_df = pd.read_csv(f'{data_folder}/standard costs.csv')

costs_df.head()

costs_df = pd.read_csv(f'{data_folder}/standard costs.csv', sep='\t')

costs_df.head()

costs_df = pd.read_csv(f'{data_folder}/standard costs.csv', sep='\t')
costs_df = costs_df.rename(columns={'Standard Unit Cost': 'Unit Cost'})
costs_df = costs_df[['ProductID', 'Unit Cost']]

costs_df.head()

costs_df.info()

# Define a function to read sales data from an Excel file:
# def get_sales(file_path):
#   df = pd.read_excel(file_path)
#   return df

# Specify the folder path where your Excel file is located:
# data_folder = r''

# Create the complete file path by combining the folder and the file name:
# file_name = 'Q1Sales.xlsx'
# file_path = f'{data_folder}\\{file_name}'

# Call the function to read the sales data:
# sales_df = get_sales(file_path)

# Sales data:
def get_sales(file_name):
    df = pd.read_(f'{data_folder}/{file_name}')
    return df

get_sales('Q1Sales.xlsx')
# Per page 359 in the book, the following code adds the other sheets below the initial dataframe in sheet/tab order:
def get_sales(file_name):
    df = pd.concat(
        pd.read_excel(f'{data_folder}/{file_name}', sheet_name=None),
        ignore_index=True
    )
    return df

# When you pass the sheet_name=None keyword argument to read_ excel, instead of a single you get a dictionary mapping sheet names to objects.
# You can concatenate the objects in this dictionary into a single by using the function as above.
# Now, if you call the get_sales function again, you will get data from all the sheets in “Q1Sales.xlsx” in a single DataFrame, per the following:
sales_df = get_sales('Q1Sales.xlsx')
sales_df.head()

sales_df.nunique()

sales_df.isna().sum()

sales_df.duplicated().sum()

def get_sales(file_name):
    df = pd.concat(
        pd.read_excel(f'{data_folder}/{file_name}', sheet_name=None),
        ignore_index=True
    )

    df = df.drop_duplicates()
    df = df.convert_dtypes()

    df = df[['InvoiceNo','Channel','ProductID','Date','Unit Price','Quantity','Total']]

# df is the original dataframe file and products_df is the one that is being added onto the original df.
    df = df.merge(products_df, on='ProductID', how='inner')
    df = df.merge(costs_df, on='ProductID', how='inner')
    df = df.sort_values('InvoiceNo', acsending=True)

    return df

sales_df = get_sales('Q1Sales.xlsx')
sales_df.head()

def get_sales(file_name):
    # Reads Q1Sales.xlsx Excel file, cleans and merges it with products and costs data, and returns a new DataFrame:
    df = pd.concat(
        pd.read_excel(f'{data_folder}/{file_name}', sheet_name=None),
        ignore_index=True
    )

    df = df.drop_duplicates()
    df = df.convert_dtypes()

# Keeps useful columns:
    df = df[['InvoiceNo','Channel','ProductID','Date','Unit Price','Quantity','Total']]

# Merges df with products_df and costs_df data:
    df = df.merge(products_df, on='ProductID')
    df = df.merge(costs_df, on='ProductID')
    df = df.sort_values('InvoiceNo', acsending=True)

# Check for missing values in any of the columns and for sale events spilling across quarters/months:
    assert df.isna().sum().sum() == 0, 'Data contains N/A values'
    assert len(df['Date'].dt.quarter.unique()) == 1, 'Data from multiple quarters/months'

    return df

# List of Excel file names to import:
excel_files = ['Q1Sales.xlsx','Q2Sales.xlsx','Q3Sales.xlsx','Q4Sales.xlsx']

sales_df = pd.DataFrame()

# For Loop through the Excel files and concatenate them together:
for file in excel_files:
    # Read each Excel file and append it to the sales_df DataFrame:
    df = pd.read_excel(file)
    sales_df = pd.concat([sales_df, df], ignore_index=True)


# os.listdir(data_folder)
# sales_df = pd.concat(
#   [get_sales(name) for name in os.listdir(data_folder) if name.endswith('.xlsx')],
#   ignore_index=True
# )
sales_df

# Formulas/Calculations:
sales_df['Gross Profit'] = (sales_df['Total'] - (sales_df['Quantity'] * sales_df['Unit Cost']))

sales_df['Profit per Unit'] = sales_df['Gross Profit'] / sales_df['Quantity']
sales_df['Margin per Unit'] = ((sales_df['Profit per Unit'] / sales_df['Unit Price']) * 100)
# Save as Excel:
sales_df.to_excel(f'{data_folder}/Sales 2023.xlsx', index=False)

# Finding answers:
import pandas as pd
import seaborn as sns
from common_code import data_folder

sales_df = pd.read_excel(f'{data_folder}/Sales 2023.xlsx')

# Channel Profits:
# 1. Which of the Channels have the highest Gross Profit?
# 2. Which of the Channels have the highest average Margin per Unit?
channel_profits_df = (
    sales_df
    .groupby('Channel')
    .agg({
        'Quantity': 'sum',
        'Gross Profit': 'sum',
        'Margin per Unit': 'mean'
    })
    .round(2)
    .sort_values('Gross Profit', ascending=True)
)

channel_profits_df

# Save as Excel:
channel_profits_df.to_excel(f'{data_folder}/Channel Profits.xlsx', index=False)

# Look to see the relationship between the highest Gross Profit and Quantity to determine if Gross Profit or Quantity is the driver of products.csv.
channel_profits_df.plot(
    kind='bar', figsize=(15, 4), subplots=True,
    layout=(1, 3), legend=False, rot=30
);

# Category Profits:
# 1. Which of the Categorys have the highest Gross Profit?
# 2. Which of the Categorys have the highest average Margin per Unit?
category_profits_df = (
    sales_df
    .groupby('Category')
    .agg({
        'Quantity': 'sum',
        'Gross Profit': 'sum',
        'Margin per Unit': 'mean'
    })
    .round(2)
    .sort_values('Gross Profit', ascending=False)
)

category_profits_df

# Save as Excel:
category_profits_df.to_excel(f'{data_folder}/Category Profits.xlsx', index=False)

# Look to see the relationship between the highest Gross Profit and Quantity to determine if Gross Profit or Quantity is the driver of sales.
category_profits_df.plot(
    kind='barh', figsize=(15, 6), subplots=True,
    layout=(1, 3), legend=False, sharex=False, sharey=True
)

# Channel Profits and Category Profits:
# Are there differences between average Margin per Unit across the Channels?
average_margin_per_category = pd.pivot_table(
    sales_df,
    index='Channel',
    columns='Category',
    values='Margin per Unit',
    aggfunc='mean'
).round(2)

average_margin_per_category
# Look at the lowest and the highest average Margin per Unit values for the answer.
sns.heatmap(
    data=average_margin_per_category.transpose(),
    cmap='Reds',
    annot=True,
    fmt=".2f",
    linewidths=.5,
    cbar=False
)

# Product Profits:
def get_product_profits(df, category):
    return df[df['Category'] == category]

get_product_profits(sales_df, 'Toys & Games')

def get_product_profits(df, category):

    df = df[df['Category'] == category]

    return (
        df.groupby('ProductID')
          .agg({
                'Product Name': 'first',
                'Unit Price': 'first',
                'Category': 'first',
                'Quantity': 'sum',
                'Gross Profit': 'sum',
                'Margin per Unit': 'mean',
            })
          .sort_values(by='Gross Profit', ascending=False)
          .reset_index()
          .round(2)
    )

get_product_profits(sales_df, 'Toys & Games')

def get_product_profits(df,
                       category='All', channel='All',
                        sort_column='Gross Profit', ascending=False):

    if category != 'All':
        df = df[df['Category'] == category]

    if channel != 'All':
        df = df[df['Channel'] == channel]

    return (
        df.groupby('ProductID')
          .agg({
                'Product Name': 'first',
                'Unit Price': 'first',
                'Category': 'first',
                'Quantity': 'sum',
                'Gross Profit': 'sum',
                'Margin per Unit': 'mean',
            })
          .sort_values(by='Gross Profit', ascending=True)
          .reset_index()
          .round(2)
    )

get_product_profits(sales_df,
                    category='Toys & Games',
                    channel='iBay.com',
                    sort_column='Margin per Unit', ascending=True)

with pd.ExcelWriter(f'{data_folder}/Product profits 2023.xlsx') as writer:
    for category in sales_df['Category'].unique():
        products_df = get_product_profits(sales_df, category)
        products_df.to_excel(writer, sheet_name=Category, index=False)
# The above Excel file will portray the highest & lowest Gross Profit and the highest & lowest Margin per Unit values for each Category.

# Sharing results and exporting notebooks:
!jupyter-nbconvert "03 - Finding answers.ipynb" --no-input --output "Analysis.html";

# Creating an interactive dashboard:
import pandas as pd

import hvplot.pandas
import panel as pn
pn.extension();

from common_code import data_folder, get_product_profits

sales_df = pd.read_excel(f'{data_folder}/Sales 2023.xlsx')

def products_scatterplot(category='All', channel='All'):
    df = get_product_profits(sales_df, category, channel)

    return df.hvplot(
        kind='scatter',
        x='Gross Profit',
        y='Margin per Unit',
        size='Quantity',
        color='Category',
        scale=0.2,
        grid=True,
        line_color='black',
        width=900,
        height=600,
        hover_cols=['ProductID', 'Product Name', 'Unit Price']
    )

products_scatterplot()

test_dropdown = pn.widgets.Select(name='Test dropdown', value='a', options=['a', 'b', 'c'])

test_dropdown

channel = ['All'] + sorted(sales_df['Channel'].unique())
channel_dropdown = pn.widgets.Select(name='Channel', value='All', options=channel)

channel_dropdown

category = ['All'] + sorted(sales_df['Category'].unique())
category_dropdown = pn.widgets.Select(name='Category', value='All', options=category)

category_dropdown

@pn.depends(category_dropdown, channel_dropdown)
def products_scatterplot(category='All', channel='All'):
    df = get_product_profits(sales_df, category, channel)

    return df.hvplot(
        kind='scatter',
        x='Gross Profit',
        y='Margin per Unit',
        size='Quantity',
        color='Category',
        scale=0.2,
        grid=True,
        line_color='black',
        width=900,
        height=600,
        hover_cols=['ProductID', 'Product Name', 'Unit Price']
    )

pn.Column(
    category_dropdown,
    channel_dropdown,
    products_scatterplot
)

dashboard = pn.Column(
    "# Products dashboard",
    "Select sales channel or product category using the menus below:",
    category_dropdown,
    channel_dropdown,
    products_scatterplot
)

dashboard.show()

dashboard.save('Products Dashboard.html', embed=True)